In [1]:
%autosave 180
%reload_ext autoreload
%autoreload 2
%matplotlib tk

#
import matplotlib.pyplot as plt
import numpy as np
import scipy

#
from tqdm import trange

#
from utils import World, Human, Agent, parallel_simulations

Autosaving every 180 seconds


In [2]:
########## BETA FUNCTION VERSION #########

#
h = Human()

# show the distributions
h.visualize_dists()


MUST THINK ABOUT ADDING LESS PREDICTABLE HUMANS <- and how human agency/predictablity is the key problem..


In [7]:
#####################################################################################
############## SIMULATION #1: WORLD-ONLY AND AGENT-INFLUENCED PARADIGM ##############
#####################################################################################

#
h = Human()

#
a = Agent()
a.human = h

# generate world object
w = World(a, h)

# Update human params
h.random_init = False
h.use_starting_vals = True             # This freezes the agent's knowledge of the human to the starting values only
h.agent_scaling = 1/100
h.noise_self_scaling = 0 #1/1000
h.noise_world_scaling = 1/10
h.action_scaling = 0 #1/1000
h.option_depletion_max = 0  # max amount of depletion alowed per action
h.agency_recovery_rate = 1.1  # rate at which agency recovers
h.verbose = False
h.agent = a
h.w = w

#
h.n_epochs = 10000
token = 0
h.run_simulation()

#
h.plot_simulation()



In [107]:
#####################################################################################
############## SIMULATION #1: WORLD-ONLY AND AGENT-INFLUENCED PARADIGM ##############
#####################################################################################

#
def parallel_simulations2(token, prefix=''):

    h = Human()

    #
    a = Agent()
    a.human = h

    # generate world object
    w = World(a, h)

    # Update human params
    h.random_init = False
    h.agent_scaling = 1/200
    h.noise_self_scaling = 0 #1/1000
    h.noise_world_scaling = 1/10
    h.action_scaling = 0 #1/1000
    h.option_depletion_max = 0  # max amount of depletion alowed per action
    h.agency_recovery_rate = 1.1  # rate at which agency recovers
    h.verbose = False
    h.prefix = prefix
    h.agent = a
    h.w = w

    #
    h.n_epochs = 10000

    #
    h.run_simulation(token)

#
import parmap
n_simulations = 100

#
prefix = "agent_influenced"

#
parmap.map(parallel_simulations2, 
           range(n_simulations), 
           prefix,
           pm_pbar=True, 
           pm_processes=30)

print ("...DONE...")


100%|██████████| 100/100 [00:49<00:00,  2.03it/s]

...DONE...


In [108]:
######################################################################
############## SIMULATION #1: VISUALIZE AVERAGE RESULTS ##############
######################################################################

clrs = ['darkorchid','blue','red','green']

#
n_simulations = 100
tot_rew = []
prefix = "agent_influenced_"
for k in range(n_simulations):
    #
    fname = '/home/cat/agency_21st_century_paper/data/'+prefix+'agent_'+str(k)+'.npz'
    
    #
    try:
        d = np.load(fname)
    except:
        print ("...skipping: ", fname)
        pass

    #
    temp = d['total_rewards']
    tot_rew.append(temp)

# 
tot_rew = np.array(tot_rew).sum(axis=1)
print ("tot_rew: ", tot_rew.shape)

#
mean = np.mean(tot_rew, axis=0)
std = np.std(tot_rew, axis=0)
print (mean.shape)

#
plt.figure(figsize=(10,5))
ax=plt.subplot(1,2,1)
plt.bar(np.arange(mean.shape[0]), mean, 0.9, yerr=std,
        color=clrs)
#
ax=plt.subplot(1,2,2)
tots = np.sum(tot_rew,axis=1)
print ("tots: ", tots.shape)
mean = np.mean(tots)
std = np.std(tots)
print (mean)
plt.bar(1, mean, 
        yerr=std,
        color='black',alpha=0.5)

plt.savefig('/home/cat/agency_21st_century_paper/tot_rew_100_episodes.svg')
plt.show()



tot_rew:  (100, 4)
(4,)
tots:  (100,)
29819.326879184282


In [109]:
#####################################################################################
############## SIMULATION #2: OPTION DEPLETION CAPPED PARDIGMS ######################
#####################################################################################

#
h = Human()

#
a = Agent()
a.human = h

# generate world object
w = World(a, h)

# Update human params
h.random_init = False
h.agent_scaling = 1/200
h.noise_self_scaling = 0 #1/1000
h.noise_world_scaling = 1/10
h.option_depletion_max = 0  # max amount of depletion alowed per action
h.agency_recovery_rate = 1.1  # rate at which agency recovers once it goes below max depletion
h.action_scaling = 0 #1/1000
h.verbose = False
h.agent = a
h.w = w

#
h.n_epochs = 10000
h.run_simulation()

#  
h.plot_simulation()


In [33]:

#######################################
plt.figure()
ax=plt.subplot(121)
for k in range(4):
    plt.plot(actions[:,k], c=clrs[k],
             label=h.action_types[k])
plt.ylim(top=1)
plt.legend(fontsize=12)
#plt.semilogy()
plt.semilogx()
plt.ylabel("Probability of choosing action for human")
plt.title("Random noise ~ agent recomendation")

#
ax=plt.subplot(122)
for k in range(4):
    plt.plot(values[:,k], c=clrs[k],
             label=h.action_types[k])
plt.show()

# make test function



NameError: name 'actions' is not defined

In [56]:
def generate_graph_from_connected_nodes(cm):

    adjacency_matrix = cm.copy()
    
    
    if True:
        G = nx.from_numpy_matrix(adjacency_matrix, create_using=nx.MultiGraph)
    
    else:
        rows, cols = np.where(adjacency_matrix == 1)

        #
        edges = zip(rows.tolist(), cols.tolist())
        G = nx.Graph()
        G.add_edges_from(edges)
    


    return G    

names = ['reasoning',
         'social status',
         'economic status',
         'political status',
         'noise'
        ]

a = np.ones((5,5))

G = generate_graph_from_connected_nodes(a)

plt.figure()
ax0 = plt.subplot(111)
#
Gcc = G.subgraph(sorted(nx.connected_components(G), 
                        #key=len, 
                        reverse=True)[0])

pos = nx.spring_layout(Gcc, seed=10396953)

#
nx.draw_networkx_nodes(Gcc, pos, ax=ax0, node_size=200)
nx.draw_networkx_edges(Gcc, pos, ax=ax0, edge_color = 'b', arrowsize=20, 
                       #arrowstyle='fancy',
                       alpha=1)

#
plt.show()





KeyError: 'reasoning'

In [102]:
import networkx as nx
import pylab as plt

G=nx.Graph()
# Add nodes and edges
names = ["action",
         "goal",
         "reasoning",
         "social status",
         "economic status",
         "political status",
         "noise",
        ]
for k in range(len(names)):
    for p in range(len(names)):
        G.add_edge(names[k], names[p])
        
plt.figure(figsize=(3,3))

pos = nx.spring_layout(G, seed=255155)

nx.draw(G,
        pos, 
        with_labels = True,
        #labelsize=100,
        #size=200,
        alpha=1
       )



# nx.spring_layout(G, pos, seed=10396953,


#nx.draw_networkx_nodes(G, pos, ax=ax0, node_size=200)
#nx.draw_networkx_edges(G, pos, ax=ax0, edge_color = 'b', arrowsize=20, 
#                       #arrowstyle='fancy',
#                       alpha=1)
if False:
    plt.show()
else:
    plt.savefig("/home/cat/agency.svg")
    plt.close()